In [13]:
import pandas as pd

# Load the CSV file
data = pd.read_csv("GOLD H1.csv")

In [14]:
# Ensure the data has 'Date', 'Time', 'Open', 'High', 'Low', 'Close' columns
# Combine Date and Time into a single datetime column
# data['datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'])
data.set_index('Date', inplace=True)

In [15]:
# Convert prices to numeric in case they are read as strings
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')

In [16]:
# Calculate Moving Averages (MA 2, MA 5, and MA 100)
data['MA_2'] = data['Close'].rolling(window=2).mean()
data['MA_5'] = data['Close'].rolling(window=5).mean()
data['MA_100'] = data['Close'].rolling(window=100).mean()

In [17]:
# Generate buy/sell signals
signals = []
for i in range(1, len(data)):
    row = data.iloc[i]
    prev_row = data.iloc[i - 1]

    if row['Close'] > row['MA_100']:
        # Long condition: Price above MA 100 and MA 2 crosses above MA 5
        if prev_row['MA_2'] < prev_row['MA_5'] and row['MA_2'] > row['MA_5']:
            signals.append((row.name, 'buy', row['Close']))
    elif row['Close'] < row['MA_100']:
        # Short condition: Price below MA 100 and MA 2 crosses below MA 5
        if prev_row['MA_2'] > prev_row['MA_5'] and row['MA_2'] < row['MA_5']:
            signals.append((row.name, 'sell', row['Close']))

In [18]:
# Calculate profits and losses
results = []
tp_pips = 60
sl_pips = 30

In [19]:
for signal in signals:
    entry_time, action, entry_price = signal
    if action == 'buy':
        tp = entry_price + tp_pips * 0.01  # Adjust based on pip size
        sl = entry_price - sl_pips * 0.01
    elif action == 'sell':
        tp = entry_price - tp_pips * 0.01
        sl = entry_price + sl_pips * 0.01

    # Track profit/loss outcome
    outcome = {'entry_time': entry_time, 'action': action, 'entry_price': entry_price}
    results.append(outcome)

In [39]:
# Convert results to a DataFrame for easier analysis
results_df = pd.DataFrame(results)

# Calculate total profits and display
# total_profit = results_df['profit'].sum()
total_profit = results_df['entry_price'].sum()
print("Total Profit:", int(total_profit/100))

Total Profit: 45103
